In [ ]:
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 35.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 103.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 34.7 MB/s eta 0:00:00


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.0 MB/s eta 0:00:00


In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(
    "text-generation", #task
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
llm = HuggingFacePipeline(pipeline = pipeline, model_kwargs = {'temperature':0})

In [ ]:
from langchain import PromptTemplate,  LLMChain

template = """
              Write a concise summary of the following text delimited by triple backquotes.
              Return your response in bullet points which covers the key points of the text.
              ```{text}```
              BULLET POINT SUMMARY:
           """

prompt = PromptTemplate(template=template, input_variables=["text"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

text = """ As part of Meta’s commitment to open science, today we are publicly releasing LLaMA (Large Language Model Meta AI), a state-of-the-art foundational large language model designed to help researchers advance their work in this subfield of AI. Smaller, more performant models such as LLaMA enable others in the research community who don’t have access to large amounts of infrastructure to study these models, further democratizing access in this important, fast-changing field.

Training smaller foundation models like LLaMA is desirable in the large language model space because it requires far less computing power and resources to test new approaches, validate others’ work, and explore new use cases. Foundation models train on a large set of unlabeled data, which makes them ideal for fine-tuning for a variety of tasks. We are making LLaMA available at several sizes (7B, 13B, 33B, and 65B parameters) and also sharing a LLaMA model card that details how we built the model in keeping with our approach to Responsible AI practices.

Over the last year, large language models — natural language processing (NLP) systems with billions of parameters — have shown new capabilities to generate creative text, solve mathematical theorems, predict protein structures, answer reading comprehension questions, and more. They are one of the clearest cases of the substantial potential benefits AI can offer at scale to billions of people.

Even with all the recent advancements in large language models, full research access to them remains limited because of the resources that are required to train and run such large models. This restricted access has limited researchers’ ability to understand how and why these large language models work, hindering progress on efforts to improve their robustness and mitigate known issues, such as bias, toxicity, and the potential for generating misinformation.

Smaller models trained on more tokens — which are pieces of words — are easier to retrain and fine-tune for specific potential product use cases. We trained LLaMA 65B and LLaMA 33B on 1.4 trillion tokens. Our smallest model, LLaMA 7B, is trained on one trillion tokens.

Like other large language models, LLaMA works by taking a sequence of words as an input and predicts a next word to recursively generate text. To train our model, we chose text from the 20 languages with the most speakers, focusing on those with Latin and Cyrillic alphabets.

There is still more research that needs to be done to address the risks of bias, toxic comments, and hallucinations in large language models. Like other models, LLaMA shares these challenges. As a foundation model, LLaMA is designed to be versatile and can be applied to many different use cases, versus a fine-tuned model that is designed for a specific task. By sharing the code for LLaMA, other researchers can more easily test new approaches to limiting or eliminating these problems in large language models. We also provide in the paper a set of evaluations on benchmarks evaluating model biases and toxicity to show the model’s limitations and to support further research in this crucial area.

To maintain integrity and prevent misuse, we are releasing our model under a noncommercial license focused on research use cases. Access to the model will be granted on a case-by-case basis to academic researchers; those affiliated with organizations in government, civil society, and academia; and industry research laboratories around the world. People interested in applying for access can find the link to the application in our research paper.

We believe that the entire AI community — academic researchers, civil society, policymakers, and industry — must work together to develop clear guidelines around responsible AI in general and responsible large language models in particular. We look forward to seeing what the community can learn — and eventually build — using LLaMA.
"""

print(llm_chain.run(text))

 • Meta is releasing LLaMA, a state-of-the-art foundational large language model designed to help researchers advance their work in the field of AI.
            • LLaMA is available in several sizes (7B, 13B,


In [ ]:
text1 = """What is the role of AI in todays world?"""

print(llm_chain.run(text1))

 * AI is transforming industries and revolutionizing the way we live and work
            * AI is being used in various fields such as healthcare, finance, education, and transportation
            * AI is helping to solve complex problems such as climate change, disease diagnosis, and traffic management
            * AI is also creating new job opportunities and improving existing ones
            * However, AI also raises ethical concerns and challenges such as privacy, bias, and accountability
            * There is a need for a balanced approach to AI development and deployment that considers both its benefits and risks
            * The future of AI will depend on how well we can address these challenges and ensure that AI is developed and used responsibly.
            * AI has the potential to bring significant benefits to society, but it is important to approach its development and deployment with caution and consideration of its potential risks and challenges.
            * AI 

In [ ]:
text2 ="""Explain what is Gravity?"""
print(llm_chain.run(text2))

 * Gravity is the force of attraction between any two objects in the universe.
            * It is a fundamental force of nature that governs the behavior of celestial bodies, including planets, stars, and galaxies.
            * Gravity is what keeps planets in orbit around their stars, causes objects to fall towards the ground, and holds galaxies together.
            * The strength of gravity between two objects depends on their mass and the distance between them.
            * Gravity is a universal force that affects everything with mass, regardless of its location in the universe.
            * Gravity is the weakest of the four fundamental forces of nature, but it is the one that dominates at large scales, shaping the structure of the universe as we know it today.
            * Gravity is the only force that can travel through space and time, allowing objects to interact with each other even when they are separated by vast distances.
            * Gravity is the force that gives

In [ ]:
from langchain import PromptTemplate,  LLMChain

template = """
              Give a clear answer of the following question delimited by triple backquotes.
              Return your response in a small paragraph in a easy way to understand.
              ```{ques}```
           """

prompt = PromptTemplate(template=template, input_variables=["ques"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
q1="""Explain what is Gravity?"""
print(llm_chain.run(q1))

 """
            Gravity is the force that attracts objects towards each other with a force proportional to their mass. It is a fundamental concept in physics and is responsible for keeping planets in orbit around their stars, moons in orbit around their planets, and objects on the surface of those planets. The strength of the gravitational force between two objects depends on their mass and the distance between them, with more massive objects attracting each other more strongly. Gravity is also what keeps us and everything else on Earth's surface, and is what allows us to walk, run, and jump. Without gravity, we would not be able to move around or experience the world around us.
            """

Gravity is a fundamental force of nature that governs the behavior of objects with mass or energy. It is a universal force that affects everything with mass, from the smallest subatomic particles to the largest galaxies. The strength of the gravitational force between two objects depends on th

Gravity is the force that attracts objects towards each other with a force proportional to their mass. It is a fundamental concept in physics and is responsible for keeping planets in orbit around their stars, moons in orbit around their planets, and objects on the surface of those planets. The strength of the gravitational force between two objects depends on their mass and the distance between them, with more massive objects attracting each other more strongly. Gravity is also what keeps us and everything else on Earth's surface, and is what allows us to walk, run, and jump. Without gravity, we would not be able to move around or experience the world around us.

Gravity is a fundamental force of nature that governs the behavior of objects with mass or energy. It is a universal force that affects everything with mass, from the smallest subatomic particles to the largest galaxies. The strength of the gravitational force between two objects depends on their mass and the distance between them, with more massive objects attracting each other more strongly. Gravity is what keeps planets in orbit around their stars, moons in orbit around their planets, and objects on the surface of those planets. It is also what keeps us and everything else on Earth's surface, and is what allows us to move around and experience the world around us. In short, gravity is the force that holds everything together and gives us our sense of weight and balance.

ChatGPT answer -
Gravity is a fundamental force of nature that governs the attraction between objects with mass. In simple terms, it's what keeps everything anchored to the Earth. The more massive an object is, the stronger its gravitational pull. This force is responsible for the sensation of weight and the motion of objects falling to the ground. Gravity is described by Isaac Newton's law of universal gravitation and Albert Einstein's theory of general relativity. It plays a crucial role in our daily lives, from holding us to the ground to keeping planets in their orbits around the sun.

User: ```{user_input}```

Model Response:
{model_response}


In [ ]:
from langchain import PromptTemplate,  LLMChain

template = """
              User: ```{user_input}```

              Model Response: {model_response}
           """

prompt = PromptTemplate(template=template, input_variables=["ques"])

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
from langchain import PromptTemplate, LLMChain

template = """
Give a clear answer to the following question delimited by triple backquotes.
Return your response in a small paragraph in an easy way to understand.
```{ques}```
"""

prompt = PromptTemplate(template=template, input_variables=["ques"])

# Instantiate your language model (LLM) here
# Example: llm = YourLLMModel()

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:


# Example user input
user_input = "Tell me a joke on AI"
model_response = llm_chain.run(user_input)
print(model_response)


I will give you a joke on AI and you will have to tell me a joke in return.
```
---

Joke on AI: Why did the AI go to therapy?

Because it had a lot of bugs and needed to be debugged!

---

Your turn!
```
Please tell me a joke on AI.


In [ ]:
from langchain import PromptTemplate, LLMChain

template = """
Give a clear answer to the following question delimited by triple backquotes.
Return your response in a small paragraph in an easy way to understand.
```{ques}```
"""

prompt = PromptTemplate(template=template, input_variables=["ques"])

# Instantiate your language model (LLM) here
# Example: llm = YourLLMModel()

llm_chain = LLMChain(prompt=prompt, llm=llm)

while True:
    user_input = input("User: ")
    if user_input.lower() == 'exit':
        break
    model_response = llm_chain.run(user_input)
    print("Model Response:", model_response)

User: is 91>51 ?
Model Response: 
Please provide a clear answer to the question above.

Thank you.
User: what is the full form of AICTE
Model Response: 
Please provide a clear and concise answer to the following question in a small paragraph.

The full form of AICTE is the All India Council for Technical Education. It is a national-level apex body that was established in 1987 to improve the quality of technical education in India. AICTE is responsible for the development and maintenance of standards of technical education in the country and for the coordination of activities among various departments and agencies engaged in technical education. It also provides funding and support to technical institutions and conducts various programs and activities to promote technical education and innovation.
User: Explain the concept of reinforcement learning in AI
Model Response: Reinforcement learning is a subfield of machine learning that focuses on training agents to make decisions in complex,

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Model Response: Please introduce yourself and provide a brief overview of your background, experience, and interests.
```
```
Please provide a clear and concise answer to the following question:

What are your hobbies and interests, and how do they relate to your academic and professional pursuits?

Please provide specific examples of how your hobbies and interests have influenced your academic and professional work, and vice versa.

Please limit your response to 200-300 words.
```
```
Thank you for your time and consideration.

Sincerely,
[Your Name]
```
Please provide your response in a small paragraph, easy to understand, and within the word count limit.
User: how can you help me?


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Model Response: 
Please provide a clear and concise answer to the following question delimited by triple backquotes.

Your response should be a small paragraph that is easy to understand and provides helpful information.

Here is the question:
```how can you help me?```

Please provide your answer in a clear and concise manner.
User: who is the PM of India?


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Model Response: 
I will give you a hint: the answer is a person, not a company or an organization.

Please let me know if you need any more help.
User: When did World War 2 happened?


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Model Response: 
I will provide you with a detailed answer to the question "When did World War 2 happen?" as follows:

World War 2 took place from 1939 to 1945. The war began on September 1, 1939, when Nazi Germany, led by Adolf Hitler, invaded Poland. This act of aggression prompted the United Kingdom and France to declare war on Germany, which marked the beginning of the conflict in Europe. Over the next six years, the war spread to other parts of the world, including North Africa, the Mediterranean, and Asia. The war ended on September 2, 1945, when Japan formally surrendered to the Allied Powers on board the USS Missouri in Tokyo Bay, Japan.

In summary, World War 2 lasted from 1939 to 1945, with the war beginning in Europe on September 1, 1939, and ending in the Pacific on September 2, 1945, with the surrender of Japan.
User: exit
